In [20]:
# NGSolve demo:
# - build a square mesh split into 2 triangles
# - set gfu via GridFunction.Set(sin(x+y))
# - build our own *elementwise (local) L2 projection* onto the same FE space
# - compare the two coefficient vectors

from ngsolve import *
from netgen.geom2d import SplineGeometry
import numpy as np
from ngsolve.webgui import Draw

# -----------------------------
# 1) Mesh: unit square -> 2 triangles
# -----------------------------
geo = SplineGeometry()
p0 = geo.AppendPoint(0,0)
p1 = geo.AppendPoint(1,0)
p2 = geo.AppendPoint(1,1)
p3 = geo.AppendPoint(0,1)

# boundary segments (domain=1)
geo.Append(["line", p0, p1], leftdomain=1, rightdomain=0, bc="bottom")
geo.Append(["line", p1, p2], leftdomain=1, rightdomain=0, bc="right")
geo.Append(["line", p2, p3], leftdomain=1, rightdomain=0, bc="top")
geo.Append(["line", p3, p0], leftdomain=1, rightdomain=0, bc="left")

ngmesh = geo.GenerateMesh(maxh=0.5)  # coarse; we'll enforce 2 triangles below
mesh = Mesh(ngmesh)

In [21]:
Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [27]:
# enforce exactly two triangles by doing a single diagonal split:
# Netgen may already triangulate into 2 triangles for maxh=2, but to be safe:
# We'll just assert size is small; if it isn't, the code still works (more elements).
print("ne (elements):", mesh.ne)

# -----------------------------
# 2) FE space + function
#    Use elementwise-discontinuous L2 space so local L2 projection is natural.
#    (H1 would need global coupling due to shared dofs.)
# -----------------------------
order = 2
fes = H1(mesh, order=order)   
gfu_set = GridFunction(fes)
gfu_proj = GridFunction(fes)

f_cf = sin(x+y)

# NGSolve's Set
gfu_set.Set(f_cf)

ne (elements): 6


In [28]:
Draw(gfu_set,mesh,'gfu')

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [29]:
print(gfu_set.vec.FV().NumPy())

[-0.00186439  0.84160185  0.90887419  0.84160185  0.47515406  0.99921095
  1.00052769  0.47681843 -0.06747071 -0.06747071 -0.18054954 -0.23246392
 -0.24632291 -0.24632291 -0.23246392 -0.18054954 -0.84474209 -0.83135998
  0.00276411 -0.00228254 -0.8190538 ]


In [30]:
# -----------------------------
# 3) Our own local L2 projection on each element:
#    For each element K:
#       M_K c_K = b_K
#       (M_K)_{ij} = ∫_K φ_i φ_j
#       (b_K)_i    = ∫_K f φ_i
# -----------------------------
u, v = fes.TnT()
mass = BilinearForm(fes, symmetric=True)
mass += u*v*dx
mass.Assemble()

rhs = LinearForm(fes)
rhs += f_cf * v * dx
rhs.Assemble()

inv_mass = mass.mat.Inverse(fes.FreeDofs(), inverse="umfpack")

gfu_proj.vec.data = inv_mass * rhs.vec

In [31]:
print(gfu_proj.vec.FV().NumPy())

[-0.0061355   0.84196841  0.91146926  0.84128997  0.47408521  0.99965607
  1.00166686  0.47624727 -0.11785575 -0.10488342 -0.17708041 -0.23253456
 -0.22538632 -0.2133216  -0.234243   -0.17788124 -0.84423374 -0.82743656
 -0.00837657  0.00596247 -0.81105548]


In [33]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
import numpy as np
from ngsolve.webgui import Draw

# -------- mesh: unit square (may have >2 triangles depending on generator)
geo = SplineGeometry()
p0=geo.AppendPoint(0,0); p1=geo.AppendPoint(1,0); p2=geo.AppendPoint(1,1); p3=geo.AppendPoint(0,1)
geo.Append(["line",p0,p1], leftdomain=1, rightdomain=0)
geo.Append(["line",p1,p2], leftdomain=1, rightdomain=0)
geo.Append(["line",p2,p3], leftdomain=1, rightdomain=0)
geo.Append(["line",p3,p0], leftdomain=1, rightdomain=0)
mesh = Mesh(geo.GenerateMesh(maxh=0.5))
Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [38]:
order = 2
fes = H1(mesh, order=order)
gfu_local = GridFunction(fes)

f_cf = sin(x+y)

# clear vector
gfu_local.vec[:] = 0

ei = ElementId(VOL, 0)
el = mesh[ei]          # or mesh[0] in some versions; we handle ElementId here
print("Element object:", type(el))
print("Available attrs:", [a for a in dir(el) if "type" in a.lower()])

Element object: <class 'ngsolve.comp.Ngs_Element'>
Available attrs: ['type']


In [39]:
# Try common element-type access patterns across versions:
if hasattr(el, "type"):
    et = el.type
elif hasattr(el, "GetType"):
    et = el.GetType()
elif hasattr(el, "ElementType"):
    et = el.ElementType()
else:
    raise RuntimeError("Cannot find element type on this NGSolve version")

print("Element type:", et)

Element type: ET.TRIG


In [35]:
intorder = 2*order + 3
ir = IntegrationRule(et, intorder)


# pick one element: first volume element
ei = ElementId(VOL, 0)
trafo = mesh.GetTrafo(ei)            # mapping reference -> physical
fel = fes.GetFE(ei)                  # local finite element on this cell
nd = fel.ndof                        # local dofs on this cell

# choose integration order (safe choice: 2*order + a bit)
intorder = 2*order + 3
# 关键：从有限元得到 element type (TRIG / QUAD / TET / ...)
ir = IntegrationRule(fel.ElementType(), intorder)


# local mass matrix and rhs
M = np.zeros((nd, nd))
b = np.zeros(nd)

for ip in ir:
    mip = trafo(ip)                  # mapped integration point (has x,y, Jac)
    w = mip.weight                   # already includes |det(J)|

    # evaluate all shape functions at this reference ip
    shapes = fel.CalcShape(ip)       # Vector of length nd

    # evaluate f at physical point
    fx = f_cf(mip)                   # CoefficientFunction is callable on mapped ip

    # accumulate
    # M_ij += ∫ phi_i phi_j
    # b_i  += ∫ f phi_i
    sh = np.array(shapes)
    M += w * np.outer(sh, sh)
    b += w * fx * sh

# solve local system
c = np.linalg.solve(M, b)

# scatter to global vector dofs for this element:
dnums = fes.GetDofNrs(ei)            # global dof numbers for this element
for j, dn in enumerate(dnums):
    if dn >= 0:
        gfu_local.vec[dn] = c[j]

print("Projected on element 0 only. ndof =", nd)

AttributeError: 'ngsolve.fem.ScalarFE' object has no attribute 'ElementType'

In [37]:
from ngsolve import *
import numpy as np

ei = ElementId(VOL, 0)
el = mesh[ei]          # or mesh[0] in some versions; we handle ElementId here
print("Element object:", type(el))
print("Available attrs:", [a for a in dir(el) if "type" in a.lower()])

# Try common element-type access patterns across versions:
if hasattr(el, "type"):
    et = el.type
elif hasattr(el, "GetType"):
    et = el.GetType()
elif hasattr(el, "ElementType"):
    et = el.ElementType()
else:
    raise RuntimeError("Cannot find element type on this NGSolve version")

print("Element type:", et)

intorder = 2*order + 3
ir = IntegrationRule(et, intorder)

Element object: <class 'ngsolve.comp.Ngs_Element'>
Available attrs: ['type']
Element type: ET.TRIG
